## 实现一个contextlib.contextmanager 装饰器

In [1]:
from contextlib import contextmanager

In [2]:
help(contextmanager)

Help on function contextmanager in module contextlib:

contextmanager(func)
    @contextmanager decorator.
    
    Typical usage:
    
        @contextmanager
        def some_generator(<arguments>):
            <setup>
            try:
                yield <value>
            finally:
                <cleanup>
    
    This makes this:
    
        with some_generator(<arguments>) as <variable>:
            <body>
    
    equivalent to this:
    
        <setup>
        try:
            <variable> = <value>
            <body>
        finally:
            <cleanup>



In [3]:
# __enter__, __exit__, biz logic

In [4]:
class ContextManager:
    def __init__(self, fn):
        self.fn = fn
        
    def __call__(self, *args, **kwargs):
        pass
    
    def __enter__(self):
        pass
    
    def __exit__(self, *args):
        pass

In [17]:
class ContextManager:
    def __init__(self, fn):
        self.fn = fn
        
    def __call__(self, *args, **kwargs):
        print('start __call__')
        self.gen = self.fn(*args, **kwargs)
        print('end __call__')
        return self
    
    def __enter__(self):
        print('__enter__')
        ret = next(self.gen)
        print('__enter__ret')
        return ret
    
    def __exit__(self, *args):
        try:
            ret = next(self.gen)
            print('__exit__ret')
            return ret
        except StopIteration as e:
            return False

In [22]:
@ContextManager
def context():
    '''this is a context'''
    print('setup')
    try:
        print('context value')
        yield 'value'
    finally:
        print('cleanup')

In [23]:
with context() as v:
    print(v)

start __call__
end __call__
__enter__
setup
context value
__enter__ret
value
cleanup


In [24]:
help(context)

Help on ContextManager in module __main__ object:

class ContextManager(builtins.object)
 |  Methods defined here:
 |  
 |  __call__(self, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __init__(self, fn)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [25]:
from functools import wraps

In [26]:
class ContextManager:
    def __init__(self, fn):
        wraps(fn)(self)
        self.fn = fn
        
    def __call__(self, *args, **kwargs):
        print('start __call__')
        self.gen = self.fn(*args, **kwargs)
        print('end __call__')
        return self
    
    def __enter__(self):
        print('__enter__')
        ret = next(self.gen)
        print('__enter__ret')
        return ret
    
    def __exit__(self, *args):
        try:
            ret = next(self.gen)
            print('__exit__ret')
            return ret
        except StopIteration as e:
            return False

In [27]:
@ContextManager
def context():
    '''this is a context'''
    print('setup')
    try:
        print('context value')
        yield 'value'
    finally:
        print('cleanup')
        
with context() as v:
    print(v)

start __call__
end __call__
__enter__
setup
context value
__enter__ret
value
cleanup


In [28]:
help(context)

Help on ContextManager in module __main__ object:

context = class ContextManager(builtins.object)
 |  Methods defined here:
 |  
 |  __call__(self, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __init__(self, fn)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [29]:
def contextmanager(fn):
    @wraps(fn)
    def wrap(*args, **kwargs):
        pass
    return wrap

In [31]:
@contextmanager
def context():
    '''this is a context'''
    print('setup')
    try:
        print('context value')
        yield 'value'
    finally:
        print('cleanup')
        


In [32]:
help(context)

Help on function context in module __main__:

context()
    this is a context



In [37]:
class ContextManager:
    def __init__(self, fn):
        self.gen = fn(*args, **kwargs)
    
    def __enter__(self):
        print('__enter__')
        ret = next(self.gen)
        print('__enter__ret')
        return ret
    
    def __exit__(self, *args):
        try:
            ret = next(self.gen)
            print('__exit__ret')
            return ret
        except StopIteration as e:
            return False

In [34]:
def contextmanager(fn):
    @wraps(fn)
    def wrap(*args, **kwargs):
        return ContextManager(fn, *args, **kwargs)
    
    return wrap

In [35]:
@contextmanager
def context():
    '''this is a context'''
    print('setup')
    try:
        print('context value')
        yield 'value'
    finally:
        print('cleanup')
        


In [36]:
help(context)

Help on function context in module __main__:

context()
    this is a context



## 实现super 函数

In [39]:
class Super:
    def __init__(type, obj):
        self.type = type
        self.obj = obj
        
class Base:
    def method(self):
        print('base')
        
class A(Base):
    def method(self):
        super(A, self).method()
        print('a')
        

In [40]:
A().method()

base
a


In [41]:
A.__mro__

(__main__.A, __main__.Base, object)

In [42]:
for cls in A.__mro__:
    if cls == A:
        print(cls)

<class '__main__.A'>


In [43]:
bases = []
append = False

for cls in A.__mro__:
    if append:
        bases.append(cls)
    if cls == A:
        append = True

In [44]:
bases

[__main__.Base, object]

In [45]:
for cls in bases:
    if hasattr(cls, 'method'):
        print(getattr(cls, 'method'))
        break

<function Base.method at 0x110f992f0>


In [48]:
getattr(A, 'method')

<function __main__.A.method>

In [49]:
from types import MethodType

In [55]:
class Super:
    def __init__(self, type, obj):
        self.type = type
        self.obj = obj
        
    # 得到方法的名字
    def lookup(self, name):
        is_super = False
        for cls in self.type.__mro__:
            if is_super and hasattr(cls, name):
                return MethodType(getattr(cls, name), self.obj)
            if cls == self.type:
                is_super = True
        raise AttributeError()
        
    def __getattr__(self, name):
        return self.lookup(name)

In [56]:
class A(Base):
    def method(self):
        Super(A, self).method()
        print('a')

In [57]:
A().method()

base
a


In [58]:
class Super:
    def __init__(self, type, obj):
        self.type = type
        self.obj = obj

    def __getattr__(self, name):
        is_super = False
        for cls in self.type.__mro__:
            if is_super and hasattr(cls, name):
                return MethodType(getattr(cls, name), self.obj)
            if cls == self.type:
                is_super = True
        raise AttributeError()
